In [1]:
import pandas as pd
from test_helpers import *

In [3]:
results = []
datasets = ['default','adult','compas']
protected_features = {'compas': 'race', 
                      'adult': 'gender',
                      'default': 'X2'
                     }
fair_met = 'EqOfOp'
test_params = {
            'price_limit': 45,
            'train_limit': 300,
            'fixed_model_params': {
                'ruleGenerator': 'Hybrid',
                'masterSolver':'barrierCrossover',
                'numRulesToReturn': 100,
                'fairness_module': 'EqOfOp'
            },
        }

eps_range = {'adult': [0.01, 0.07,0.15, 1],
    'compas': [0.01, 0.2, 1],
     'default': [0.01, 0.05,0.1, 1]
    }
C_range = {'adult': [5, 15, 20, 30, 40],
    'compas': [5, 10, 15, 20, 30],
     'default': [5, 10, 15, 20, 30]
    }

for dataset in datasets:
    print('**** DATASET %s ******'%dataset)
    group_var = protected_features[dataset]
    name = dataset+'_'+str(fair_met)

    for i in range(10):
        print('***** FOLD %d ******'%i)
        train  = pd.read_csv('data_split/bin_'+dataset+'_train_%d.csv'%i)
        test = pd.read_csv('data_split/bin_'+dataset+'_test_%d.csv'%i)
        
        saved_rules = None
        for eps in eps_range[dataset]:
            for C in C_range[dataset]:
                res = TestResults(name+' '+'(%d,%d)'%(eps, C)+'-'+str(i))
                res.res['eps'] = eps
                res.res['C'] = C

                test_params = test_params.copy()
                test_params['fixed_model_params']['epsilon'] = eps
                test_params['fixed_model_params']['ruleComplexity'] = C

                res, classif = runSingleTest(train.drop('Y',axis=1).to_numpy(), train['Y'].to_numpy(), 
                                             train[group_var].to_numpy(), 
                                             test.drop('Y',axis=1).to_numpy(), test['Y'].to_numpy(), 
                                             test[group_var].to_numpy(), 
                                             test_params, 
                                             saved_rules, res, colGen = True, rule_filter = False)
                
                rules = classif.ruleMod.rules
                saved_rules = updateRuleSet(saved_rules,rules)
        np.save('rules/'+name+'_fold_'+str(i)+'_rules.txt', saved_rules)

**** DATASET default ******
***** FOLD 0 ******
Using Hybrid

--------------------------------------------
--------------------------------------------

Using license file /Users/connorlawless/gurobi.lic
Academic license - for non-commercial use only
Using barrier with default crossover
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Time limit for column generation exceeded. Solving MIP.
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


/Users/connorlawless/opt/anaconda3/envs/CourseSchedule/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/connorlawless/opt/anaconda3/envs/CourseSchedule/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/connorlawless/opt/anaconda3/envs/CourseSchedule/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/connorlawless/opt/anaconda3/envs/CourseSchedule/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/connorlawless/opt/anaconda3/envs/CourseSchedule/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount

Results: (Acc) 0.783 (Time) 300 (Complex) 0 (Cols Gen) 364  (Num Iter) 194
{'accuracy': [0.7833333333333333], 'accuracy_train': [0.7782962962962963], 'mip': [[5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5986.0, 5

KeyboardInterrupt: 

In [4]:
results = []
datasets = ['compas','default','adult']
protected_features = {'compas': 'race', 
                      'adult': 'gender',
                      'default': 'X2'
                     }
fair_met = 'HammingEqOdd'
test_params = {
            'price_limit': 45,
            'train_limit': 300,
            'fixed_model_params': {
                'ruleGenerator': 'Hybrid',
                'masterSolver':'barrierCrossover',
                'numRulesToReturn': 100,
                'fairness_module': 'HammingEqOdd'
            },
        }

eps_range = {'adult': [0.01, 0.7,0.15, 1],
    'compas': [1, 0.01, 0.2, ],
     'default': [0.01, 0.05,0.1, 1]
    }
C_range = {'adult': [5, 15, 20, 30, 40],
    'compas': [30, 5, 10, 15, 20],
     'default': [5, 10, 15, 20, 30]
    }

for dataset in datasets:
    print('**** DATASET %s ******'%dataset)
    group_var = protected_features[dataset]
    name = dataset+'_'+str(fair_met)

    for i in range(10):
        print('***** FOLD %d ******'%i)
        train  = pd.read_csv('data_split/bin_'+dataset+'_train_%d.csv'%i)
        test = pd.read_csv('data_split/bin_'+dataset+'_test_%d.csv'%i)
        
        saved_rules = None
        for eps in eps_range[dataset]:
            for C in C_range[dataset]:
                res = TestResults(name+' '+'(%d,%d)'%(eps, C)+'-'+str(i))
                res.res['eps'] = eps
                res.res['C'] = C

                test_params = test_params.copy()
                test_params['fixed_model_params']['epsilon'] = eps
                test_params['fixed_model_params']['ruleComplexity'] = C

                res, classif = runSingleTest(train.drop('Y',axis=1).to_numpy(), train['Y'].to_numpy(), 
                                             train[group_var].to_numpy(), 
                                             test.drop('Y',axis=1).to_numpy(), test['Y'].to_numpy(), 
                                             test[group_var].to_numpy(), 
                                             test_params, 
                                             saved_rules, res, colGen = True, rule_filter = False)
                
                rules = classif.ruleMod.rules
                saved_rules = updateRuleSet(saved_rules,rules)
        np.save('rules/'+name+'_fold_'+str(i)+'_rules.txt', saved_rules)

**** DATASET compas ******
***** FOLD 0 ******
Using Greedy
Using barrier with default crossover
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1


IndexError: boolean index did not match indexed array along dimension 0; dimension is 929 but corresponding boolean dimension is 2231